In [1]:
import pandas as pd

teams = pd.read_csv('data/original/teams.csv')
players = pd.read_csv('data/original/players.csv')
coaches = pd.read_csv('data/original/coaches.csv')
players_teams = pd.read_csv('data/original/players_teams.csv')
series_post = pd.read_csv('data/original/series_post.csv')
awards_players = pd.read_csv('data/original/awards_players.csv')
teams_post = pd.read_csv('data/original/teams_post.csv')

In [2]:
# Function to get columns with only one value
def get_empty_columns(data):
    for column in data.columns:
        if data[column].nunique() == 1:
            print("Column with " + str(data[column].nunique()) + " unique values: " + column)

## Cleaning teams.csv

In [3]:
get_empty_columns(teams)

Column with 1 unique values: lgID
Column with 1 unique values: seeded
Column with 1 unique values: tmORB
Column with 1 unique values: tmDRB
Column with 1 unique values: tmTRB
Column with 1 unique values: opptmORB
Column with 1 unique values: opptmDRB
Column with 1 unique values: opptmTRB


In [4]:
# Drop the columns that are not needed

teams = teams.drop(columns=['lgID', 'franchID', 'divID', 'seeded', 'arena', 'name', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])

## Cleaning players.csv

In [5]:
get_empty_columns(players)

Column with 1 unique values: firstseason
Column with 1 unique values: lastseason


In [6]:
# Drop the columns that are not needed

players = players.drop(columns=['firstseason', 'lastseason', 'college', 'collegeOther'])


In [7]:
def convert_height_to_cm(height):
    height = int(height)
    return int(height * 2.54)

def convert_weight_to_kg(weight):
    weight = int(weight)
    return int(weight * 0.453592)

players['height'] = players['height'].apply(convert_height_to_cm)
players['weight'] = players['weight'].apply(convert_weight_to_kg)

In [8]:
# check if any ids from players are not in players_teams and drop them from players if their weight is 0

players_ids = players['bioID'].unique()
players_teams_ids = players_teams['playerID'].unique()

for player_id in players_ids:
    if player_id not in players_teams_ids:
        # Drop only if weight and height are 0 and birthDate is 0000-00-00
        if players.loc[players['bioID'] == player_id, 'weight'].values[0] == 0 and players.loc[players['bioID'] == player_id, 'height'].values[0] == 0 and players.loc[players['bioID'] == player_id, 'birthDate'].values[0] == '0000-00-00':
            players = players[players.bioID != player_id]
    

In [9]:
# Put the Outliers to 0

# Put the height values with <150 cm to 0

players.loc[players['height'] < 150, 'height'] = 0

# Put the weight values with <54 kg to 0

players.loc[players['weight'] < 50, 'weight'] = 0

In [10]:
# Fill the outliers with the average value by position

# Get the average height and weight by position

dict_pos_height = {}
dict_pos_weight = {}
for index, row in players.iterrows():
    if '-' in row['pos']:
        pos = row['pos'].split('-')
        pos1 = pos[0]
        pos2 = pos[1]
        if pos1 not in dict_pos_height:
            dict_pos_height[pos1] = []
            dict_pos_weight[pos1] = []
        if pos2 not in dict_pos_height:
            dict_pos_height[pos2] = []
            dict_pos_weight[pos2] = []
        dict_pos_height[pos1].append(row['height'])
        dict_pos_height[pos2].append(row['height'])
        dict_pos_weight[pos1].append(row['weight'])
        dict_pos_weight[pos2].append(row['weight'])
    else:
        if row['pos'] not in dict_pos_height:
            dict_pos_height[row['pos']] = []
            dict_pos_weight[row['pos']] = []
        dict_pos_height[row['pos']].append(row['height'])
        dict_pos_weight[row['pos']].append(row['weight'])

average_height_by_position = {}

for key in dict_pos_height:
    average_height_by_position[key] = int(sum(dict_pos_height[key]) / len(dict_pos_height[key]))

average_weight_by_position = {}

for key in dict_pos_weight:
    average_weight_by_position[key] = int(sum(dict_pos_weight[key]) / len(dict_pos_weight[key]))


print(average_height_by_position)
print(average_weight_by_position)


# Fill the outliers with the average value by position

for index, row in players.iterrows():
    if row['height'] == 0:
        if '-' in row['pos']:
            pos = row['pos'].split('-')
            pos1 = pos[0]
            pos2 = pos[1]
            players.at[index, 'height'] = int((average_height_by_position[pos1] + average_height_by_position[pos2]) / 2)
        else:
            players.at[index, 'height'] = average_height_by_position[row['pos']]
    if row['weight'] == 0:
        if '-' in row['pos']:
            pos = row['pos'].split('-')
            pos1 = pos[0]
            pos2 = pos[1]
            players.at[index, 'weight'] = int((average_weight_by_position[pos1] + average_weight_by_position[pos2]) / 2)
        else:
            players.at[index, 'weight'] = average_weight_by_position[row['pos']]


{'C': 191, 'F': 185, 'G': 174}
{'C': 81, 'F': 74, 'G': 65}


## Add the awards columns to the players_teams and coaches 

In [11]:

# Add the awards columns to the players_teams
players_teams['award_ASGMVP'] = 0
players_teams['award_DPOTY'] = 0
players_teams['award_KPS'] = 0
players_teams['award_MIP'] = 0
players_teams['award_MVP'] = 0
players_teams['award_ROTY'] = 0
players_teams['award_SWOTY'] = 0
players_teams['award_WFMVP'] = 0
players_teams['award_WADT'] = 0
players_teams['award_WADTHM'] = 0

# Add the award column to the coaches
coaches['award_COTY'] = 0


# Put 1 in the players that won that awards in 1 specific year, based on the awards_players table
for index, row in awards_players.iterrows():
    player_id = row['playerID']
    award_name = row['award']
    award_year = row['year']
    if award_name == 'All-Star Game Most Valuable Player':
        players_teams.loc[(players_teams['playerID'] == player_id) & (players_teams['year'] == award_year), 'award_ASGMVP'] = 1
    elif award_name == 'Defensive Player of the Year':
        players_teams.loc[(players_teams['playerID'] == player_id) & (players_teams['year'] == award_year), 'award_DPOTY'] = 1
    elif award_name == 'Kim Perrot Sportsmanship Award' or award_name == 'Kim Perrot Sportsmanship':
        players_teams.loc[(players_teams['playerID'] == player_id) & (players_teams['year'] == award_year), 'award_KPS'] = 1
    elif award_name == 'Most Improved Player':
        players_teams.loc[(players_teams['playerID'] == player_id) & (players_teams['year'] == award_year), 'award_MIP'] = 1
    elif award_name == 'Most Valuable Player':
        players_teams.loc[(players_teams['playerID'] == player_id) & (players_teams['year'] == award_year), 'award_MVP'] = 1
    elif award_name == 'Rookie of the Year':
        players_teams.loc[(players_teams['playerID'] == player_id) & (players_teams['year'] == award_year), 'award_ROTY'] = 1
    elif award_name == 'Sixth Woman of the Year':
        players_teams.loc[(players_teams['playerID'] == player_id) & (players_teams['year'] == award_year), 'award_SWOTY'] = 1
    elif award_name == 'WNBA Finals Most Valuable Player':
        players_teams.loc[(players_teams['playerID'] == player_id) & (players_teams['year'] == award_year), 'award_WFMVP'] = 1
    elif award_name == 'WNBA All-Decade Team':
        players_teams.loc[(players_teams['playerID'] == player_id) & (players_teams['year'] == award_year), 'award_WADT'] = 1
    elif award_name == 'WNBA All Decade Team Honorable Mention':
        players_teams.loc[(players_teams['playerID'] == player_id) & (players_teams['year'] == award_year), 'award_WADTHM'] = 1

# Put 1 in the coaches that won that awards in 1 specific year, based on the awards_players table
for index, row in awards_players.iterrows():
    coach_id = row['playerID']
    award_name = row['award']
    award_year = row['year']
    if award_name == 'Coach of the Year':
        coaches.loc[(coaches['coachID'] == coach_id) & (coaches['year'] == award_year), 'award_COTY'] = 1


## Add the post series winrate column to the teams.csv

In [12]:
# Add the post series winrate column to the teams.csv

teams['post_winrate'] = None

for index, row in teams_post.iterrows():
    team_id = row['tmID']
    year = row['year']
    winrate = int((row['W'] / (row['W'] + row['L'])) * 100)
    teams.loc[(teams['tmID'] == team_id) & (teams['year'] == year), 'post_winrate'] = winrate


## Cleaning coaches.csv

In [13]:
get_empty_columns(coaches)

Column with 1 unique values: lgID


In [14]:
# Drop the columns that are not needed

coaches = coaches.drop(columns=['lgID'])

## Cleaning players_teams.csv

In [15]:
# Drop the columns that are not needed

players_teams = players_teams.drop(columns=['lgID'])

## Cleaning series_post.csv

In [16]:
# Drop the columns that are not needed

#series_post = series_post.drop(columns=['lgIDWinner', 'lgIDLoser'])

In [17]:
teams.to_csv('data/clean/cleaned_teams.csv', index=False)
players.to_csv('data/clean/cleaned_players.csv', index=False)
coaches.to_csv('data/clean/cleaned_coaches.csv', index=False)
players_teams.to_csv('data/clean/cleaned_players_teams.csv', index=False)
#series_post.to_csv('data/clean/cleaned_series_post.csv', index=False)